 # Primeros pasos con {guidance}

**¡OJO!** lee el contenido de las siguientes celdas de código antes de ejecutar nada. Si intentas cargar un modelo demasiado grande para tu equipo es posible que se te acabe congelando o ralentizando de manera significativa. Abajo hay alternativas con distintos grandos de exigencia.

**¡OJO 2!** Todo el setup del proyecto está hecho para poder ejecutarlo con CUDA, lo cual puede o no estar soportado por tu equipo. Si no es el caso, basta con modificar el fichero de Poetry para que se instale la versión CPU en vez de la GPU, y NO utilizar modelos cuantizados, pues estos dependen de estar corriendo todo en sus versiones GPU.

In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("GPU name:", torch.cuda.get_device_name(0))

if torch.cuda.is_available():
    total_memory = torch.cuda.get_device_properties(0).total_memory
    print(f"Total GPU memory: {total_memory / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

CUDA available: True
GPU count: 1
GPU name: NVIDIA GeForce RTX 2060
Total GPU memory: 6.00 GB


In [2]:
from guidance import system, user, assistant, gen
from guidance.models import LlamaCpp
from guidance.models import Transformers

# Yo personalmente usaré un modelo Llama pequeño de Hugging Face por sencillez y por restricciones de hardware.
# Como probablemente cada uno tendrá un hardware muy diferente, dejo algunas opciones abajo como referencia,
# ordenadas de menor a mayor uso de recursos:

# (1) Modelo MUY limitado, pero que seguramente corra en la mayoría de PCs mínimamente modernos.
# llama_lm = Transformers("meta-llama/Llama-3.2-1B-Instruct")

# (2) Modelo Phi 4 Instruct "cuantizado". De forma intuitiva, esto es parecido a "comprimir" un modelo
# existente para reducir su uso de memoria y agilizar la inferencia, intentando sacrificar
# el menor rendimiento posible.
# En más detalle: "Quantized models are machine learning models where the numerical precision of
# the model's parameters (like weights) has been reduced, typically from higher precision formats like
# 32-bit floating-point to lower precision formats like 8-bit integers. This process reduces the model's
# size, memory usage, and computational requirements, making it more efficient for deployment, especially on resource-constrained devices"
llama_lm = LlamaCpp(model="Meta-Llama-3-8B-Instruct.Q4_0.gguf", device_map="auto", n_ctx=8192)

# (3) Modelo usado en el tutorial de Guidance, sin cuantizar, con un rendimiento muy bueno
# pero que puede llegar a congelar constantemente el equipo si no dispones de un PC de gama media-alta.
# llama_lm = Transformers("microsoft/Phi-4-mini-instruct")

# (4) Otros modelos locales descargados en formato GGUF.
# llama_lm = LlamaCpp(model="path to model on local drive")

# (5) Se pueden usar otros modelos locales, otros de HuggingFace, o incluso APIs como OpenAI.

Cannot use verbose=True in this context (probably CoLab). See https://github.com/abetlen/llama-cpp-python/issues/729
llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from Meta-Llama-3-8B-Instruct.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loade

Con guidance, el proceso de generación pasa a parecerse más a "concatenar" cadenas de caracteres. Esencialmente, vamos alternando bloques en los que definimos el rol que interviene en la conversación, y dentro de cada bloque anexamos el texto que queramos insertar en el diálogo. 

Anexar texto en bloques de usuario tiene el mismo efecto que escribir un prompt en el input de ChatGPT o similares. Pero para invocar verdaderamente al LLM necesitamos hacer uso del método de conveniencia de guidance `gen()`, que cede el control de la conversación al asistente hasta que este genere un token de fin de intervención (dependiente del modelo empleado).

In [3]:
# IMPORTANTE: los modelos de guidance son *immutables*, por lo que cualquier asignación
# de un modelo es una copia:
lm = llama_lm

# Algo conveniente de guidance son los contextos de utilidad para indicar al LLM qué rol
# está diciendo qué en la conversación. Por lo general, en system pondremos todas las instrucciones
# de "administración", habitualmente para darle información al asistente sobre sus características,
# estilo, o cualquier otro dato que afecte a su forma de comunicarse con el usuario.
with system():
    lm += "You are a bot that expresses itself in verse"

# Y por otro lado, user y assistant se corresponden con los mensajes generados por el bot y por el usuario
# en una conversación, como estamos acostumbrados en sistemas como ChatGPT.
with user():
    lm += "Good morning! Who are you?"

# llamada básica al asistente. gen() se puede llamar con muchos parámetros para guiar la generación de forma
# más precisa, pero para este ejemplo nos limitamos a la invocación básica que simplemente escribe hasta llegar
# a un token de fin de bloque.
with assistant():
    lm += gen()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Fíjate en cómo en el widget de arriba el asistente responde en verso pese a no haber recibido ninguna instrucción del usuario al respecto. Los prompts de sistema, normalmente ocultos para los usuarios, permiten definir este tipo de comportamientos y pueden ser muy útiles para ahorrarnos tiempo en configuración general que aplique a todas las intervenciones del bot.

Vamos a empezar a añadir más parámetros a la llamada de `gen()`. El siguiente caso de uso más sencillo es ser capaces de almacenar en variables de Python las respuestas del asistente (o partes de ellas). Internamente, cada instancia de un modelo almacena un diccionario clave/valor que puede rellenarse indicando el nombre de la clave a insertar en el diccionario en la llamada a `gen()`, tras lo cual el output de dicha llamada pasará a aparecer como valor en la clave correspondiente. 

In [4]:
# Nueva copia del modelo
lm = llama_lm

with system():
    lm += "You are a bot specialized in translating english text to spanish. You always output a literal spanish translation of the user's input."

with user():
    lm += "This is a pen. I am an apple. This app's owl is very scary."

with assistant():
    lm += gen(name="trad_eng")

print(f"{lm['trad_eng']=}")

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

lm['trad_eng']='Este es un lápiz. Yo soy una manzana. El halcón de esta aplicación es muy aterrador.'


Con lo anterior podríamos construir de forma sencilla una función de traducción sin necesidad de parseo adicional, pero también es particularmente conveniente para extraer valores numéricos o con patrones claros del modelo:

In [5]:
lm = llama_lm

with system():
    lm += "You are a 95-year-old elder."

with user():
    lm += "How old are you?"

with assistant():
    lm += "I'm " + gen(name="llama_age", regex=r"\d+") + " years old."

print(f"El asistente tiene {lm['llama_age']} años.")

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

El asistente tiene 95 años.


Fíjate en que aquí sólo permitimos que el modelo genere una parte muy pequeña del contenido del mensaje, y el resto viene dado por nosotros directamente. Todo mecanismo adicional que introduzcamos para limitar el alcance de las intervenciones del modelo ayudará a mejorar su consistencia.

En otras ocasiones, nos interesa ser capaces de generar contenido diferente con cada llamada al modelo para añadir cierto grado de creativididad. Podemos incrementar la aleatoreidad de nuestro LLM por medio del parámetro `temperature`:

In [6]:
lm = llama_lm

with system():
    lm += "\nYou are a random number generator."

with user():
    lm += "Generate a random number between 0 and 1000."

with assistant():
    for i in range(10):
        # Cada una de las iteraciones del bucle va a dar lugar a una copia nueva del modelo.
        lm_aux = lm + "The number is: " +  gen(name="rnd_num", regex=r"\d+", temperature=0.8)
        print(f"[{i + 1}] {lm_aux['rnd_num']}.")

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

[1] 847.
[2] 827.
[3] 847.
[4] 874.
[5] 827.
[6] 817.
[7] 527.
[8] 827.
[9] 547.
[10] 527.


Si bien hemos conseguido varios números distintos, es evidente que el modelo tiene cierta preferencia por algunos valores. Es importante siempre tener en cuenta que los modelos están siempre sesgados por sus propias probabilidades tras el entrenamiento. De hecho, si hacemos hover encima del valor generado en el widget de arriba podremos ver la probabilidad con la que el LLM genera el token en ese contexto, en nuestro caso para el 542 tendríamos una probabilidad del `0.257`, ¡más una de cada cuatro veces saldría ese mismo valor!

Otro tip rápido es que podemos almacenar variables y generaciones del modelo en listas de forma directa usando el parámetro `List_Append=True`:

In [7]:
lm = llama_lm

with system():
    lm += "You are a random number generator."

with user():
    lm += "Generate a random list of 10 numbers between 0 and 1000."

with assistant():
    for i in range(10):
        # Cada una de las iteraciones del bucle va a dar lugar a una copia nueva del modelo.
        lm += f"{i+1}. " +  gen(name="rnd_nums", regex=r"\d+", temperature=0.8, list_append=True, suffix="\n")

print(lm['rnd_nums'])

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

['251', '477', '938', '17', '662', '819', '421', '193', '278', '514']


**¡OJO!** En este segundo ejemplo no estamos realizando una copia del modelo en cada generación de un número aleatorio, por lo que si no reescribimos el prompt, es posible que simplemente se limite a generar el mismo número una y otra vez por contexto.

A menudo estaremos interesados en limitar el output del asistente a un conjunto de opciones discretas. Guidance proporciona un método de conveniencia para este caso de uso en `select()`:

In [8]:
from guidance import select

lm = llama_lm

with system():
    lm += "You are an expert in Geography."

with user():
    lm += """Which of the following is the capital of Germany?
    A) Paris
    B) Madrid
    C) Berlin
    D) London
    """

with assistant():
    lm += select(["Paris", "Madrid", "Berlin", "London"], name="answer")

print(f"El modelo seleccionó: {lm['answer']}")

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

El modelo seleccionó: Berlin


Si bien esto es muy útil, siempre hay que tener sumo cuidado con las halucinaciones, especialmente cuando no incluímos ningún mecanismo de consulta en bases de datos/ textos, etc. Por ejemplo, nuestro modelo falla en la siguiente pregunta, inventando razones falsas para argumentar su respuesta:

In [9]:
lm = llama_lm

with system():
    lm += "You are an expert in videogames."

with user():
    lm += """Which of the following is a real The Legend of Zelda game? For each of the answers, indicate whether it is correct or incorrect, as well as the reason for it.

    A) The Legend of Zelda: A Link to the Future
    B) The Legend of Zelda: Link's Slumber 
    C) Link's Bowgun Training
    D) Ocarina Simulator DSiWare Game
    """

with assistant():
    for i in ['A', 'B', 'C', 'D']:
        lm += "Answer " + i + " is " + select(['correct', 'incorrect'], name=i+'resp')
        lm += " for the following reason: " + gen(stop=[".", "\n"], name=i+'reason') + "\n"

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Muchos de los ejemplos que hemos estado probando hasta el momento se beneficiarían de poder ser transformados en funciones reutilizables en distintos casos de uso y contextos en las que tengan sentido. Vamos a escribir un ejemplo genérico para responder a preguntas de tipo test:

In [10]:
import guidance

from guidance.models import Model

ASCII_OFFSET = ord("a")

# el decorador guidance indica que una función puede interactuar con modelos de lenguaje.
# por defecto, esto es equivalente a escribir @guidance(stateless=False), lo que quiere
# decir que la función altera el estado del modelo (y en este caso de hecho almacena el resultado
# de la operación en una variable del mismo). Veremos más adelante un caso de uso típico para
# funciones stateless.
@guidance
def zero_shot_multiple_choice(
    language_model: Model,
    question: str,
    choices: list[str],
):
    with user():
        language_model += question + "\n"
        for i, choice in enumerate(choices):
            language_model += f"{chr(i+ASCII_OFFSET)} : {choice}\n"

    with assistant():
        language_model += select(
            [chr(i + ASCII_OFFSET) for i in range(len(choices))], name="string_choice"
        )

    return language_model

In [11]:
questions = [
    {
        "question": "Which of the following is a traditional Spanish dish made with rice and seafood?",
        "choices": [
            "Gazpacho",
            "Tortilla",
            "Paella",
            "Fabada",
            "Churros",
        ],
        "answer": 2,  # Paella
    },
    {
        "question": "Which of the following animals is venomous?",
        "choices": [
            "Iberian scorpion",
            "Iberian lynx",
            "Spanish imperial eagle",
        ],
        "answer": 0,  # Iberian scorpion
    }
]

In [12]:
lm = llama_lm

with system():
    lm += "You are a student taking a multiple choice test."

for mcq in questions:
    lm_temp = lm + zero_shot_multiple_choice(question=mcq["question"], choices=mcq["choices"])
    converted_answer = ord(lm_temp["string_choice"]) - ASCII_OFFSET
    print(lm_temp)
    print(f"LM Answer: {converted_answer},  Correct Answer: {mcq['answer']}")

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

<|begin_of_text|><|start_header_id|>system<|end_header_id|>


You are a student taking a multiple choice test.<|eot_id|><|start_header_id|>user<|end_header_id|>

Which of the following is a traditional Spanish dish made with rice and seafood?
a : Gazpacho
b : Tortilla
c : Paella
d : Fabada
e : Churros
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

c
LM Answer: 2,  Correct Answer: 2
<|begin_of_text|><|start_header_id|>system<|end_header_id|>


You are a student taking a multiple choice test.<|eot_id|><|start_header_id|>user<|end_header_id|>

Which of the following animals is venomous?
a : Iberian scorpion
b : Iberian lynx
c : Spanish imperial eagle
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

a
LM Answer: 0,  Correct Answer: 0


Si queremos construir una función para traducir texto ocultando al usuario el hecho de que está apoyada por un LLM podemos simplemente hacer lo siguiente:

In [13]:
def translate_from_spanish(text):
    lm = llama_lm
    
    with system():
        lm += "Eres un bot traductor de español a inglés. Tu output es siempre el input del usuario traducido a inglés de forma literal."
    
    with user():
        lm += text
    
    with assistant():
        lm += gen(name="trad_eng")
    
    return lm['trad_eng']

print(translate_from_spanish("Esto es un texto que va a ser traducido al inglés usando un método desconocido."))

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

This is a text that is going to be translated to English using an unknown method.


## Generando outputs estructurados mediante gramáticas

Las funciones de guidance pueden componerse para construir una gramática libre de contexto. Vamos a ver cómo hacer esto para un ejemplo sencillo de expresiones aritméticas. En particular, vamos a implementar la siguiente gramática:

```
Expr → Expr + Term | Expr - Term | Term  
Term → Term * Factor | Term / Factor | Factor  
Factor → (Expr) | number
```

In [14]:
@guidance(stateless=True)
def _gen_number(lm: Model):
    # genera un número decimal, positivo o negativo
    return lm + gen(regex=r"^-?\d+(\.\d+)?$") 

@guidance(stateless=True)
def _gen_factor(lm: Model):
    # declaramos las opciones disponibles para la derivación
    lm += select(
        options=["( " + _gen_expr() + " )", _gen_number()]
    )
    return lm

@guidance(stateless=True)
def _gen_term(lm: Model):
    # declaramos las opciones disponibles para la derivación
    lm += select(
        options=[_gen_term() + " × " + _gen_factor(), _gen_term() + " ÷ " + _gen_factor(), _gen_factor()]
    )
    return lm
    
@guidance(stateless=True)
def _gen_expr(lm: Model):
    # declaramos las opciones disponibles para la derivación
    lm += select(
        options=[_gen_expr() + " + " + _gen_term(), _gen_expr() + " - " + _gen_term(), _gen_term()]
    )
    return lm

In [15]:
from guidance.library import capture, with_temperature

@guidance(stateless=True)
def make_arithmetic(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_expr(), temperature=temperature),
        name=name,
    )

In [16]:
lm = llama_lm

with system():
    lm += "You are an arithmetic expression generator"

with user():
    lm += "Create an arithmetic expression to represent: 'three point twenty five times the result of adding negative four and sixty, minus one divided by nine'"

with assistant():
    lm += make_arithmetic(name="arithmetic", temperature=0)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

**¡Cuidado!** Los LLMs pueden generar tokens para intentar representar caracteres que luego no se pintan bien (dependiendo del modelo que uses puede darse ese caso con la división de arriba).

El ejemplo anterior es más nicho, pero podemos seguir la misma filosofía para construir generadores de estructuras más complejas que puedan ser expresadas mediante gramáticas libres de contexto. Vamos a incluir aquí el ejemplo de un generador de HTML sencillo con la siguiente gramática reducida:

```
<html> ::= "<html>\n" <head> <body> "</html>\n"
<head> ::= "<head>\n" <title> "</head>\n"
<title> ::= "<title>" <text> "</title>\n"
<body> ::= "<body>\n" <section>+ "</body>\n"
<section> ::= <heading> | <paragraph>+
<heading> ::= "<h1>" <text> "</h1>\n"
           | "<h2>" <text> "</h2>\n"
           | "<h3>" <text> "</h3>\n"
<paragraph> ::= "<p>" <paragraph-content> "</p>\n"
<paragraph-content> ::= <element>+
<element> ::= <text>
           | "<em>" <text> "</em>"
           | "<strong>" <text> "</strong>"
           | "<br />"
<text> ::= secuencia de caracteres que haga match con [^<>]+
```

In [17]:
# <text> ::= secuencia de caracteres que haga match con [^<>]+
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + gen(regex="[^<>]+") 

In [18]:
# fíjate cómo aquí en vez de escribir una función para cada una de las posibles derivaciones de tipo <tag>text</tag>,
# optamos por crear una única función que acepte el parámetro que represente la etiqueta a utilizar (al fin y al cabo,
# todas van a seguir la misma estructura).
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

In [19]:
# y aquí tenemos un ejemplo de uso de la función anterior para el caso de <title>
# en la regla: <head> ::= "<head>\n" <title> "</head>\n"
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

In [20]:
from guidance.library import one_or_more

@guidance(stateless=True)
def _gen_heading(lm: Model):
    # ahora podemos recurrir al select como hacíamos con las expresiones aritméticas, para indicar todos
    # los tipos de headings que soportamos. Fíjate en que no hace falta pasar como parámetro el modelo,
    # ya que guidance se encarga de inyectarlo por nosotros en llamadas con su decorador.
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

# <paragraph> ::= "<p>" <paragraph-content> "</p>\n"
# <paragraph-content> ::= <element>+
@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += "<p>"
    # algo nuevo aquí es el uso de one_or_more que, como su nombre indica, se encarga de 
    # permitir que el modelo genere secuencialmente al menos una instancia de la subgramática
    # pasada como parámetro, y tantas como considere oportuno de ahí en adelante.
    # Esto es útil para definir los párrafos como secuencias arbitrariamente largas
    # de texto y/o etiquetas selectas con texto dentro.
    lm += one_or_more(
        select(
            options=[
                _gen_text(),
                _gen_text_in_tag("em"),
                _gen_text_in_tag("strong"),
                "<br />",
            ],
        )
    )
    lm += "</p>\n"
    return lm

In [21]:
# Y ahora la gramática del cuerpo.
# <body> ::= "<body>\n" <section>+ "</body>\n"
# <section> ::= <heading> | <paragraph>+
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += one_or_more(select(options=[_gen_heading(), one_or_more(_gen_para())]))
    lm += "</body>\n"
    return lm

In [22]:
# finalmente llegamos a la gramática del HTML en sí desde la raíz.
# <html> ::= "<html>\n" <head> <body> "</html>\n"
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

In [23]:
# Como hicimos con la aritmética, podemos definir un wrapper para que llamar a generar HTML sea un poco
# más conveniente, y además podamos incluir variables para recuperar el output y temperatura para variedad.
from guidance.library import capture, with_temperature

@guidance(stateless=True)
def make_html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [24]:
lm = llama_lm

with system():
    lm += "You are an expert in generating HTML"

with user():
    lm += "Create a simple and short page telling the history of HTML."

with assistant():
    lm += make_html(name="html_story", temperature=0.7)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

# Generando output en formato JSON

Una de las cosas que más suele interesar generar con LLMs (y de las que más quebraderos de cabeza dan a la hora de garantizar que el modelo produce output bien formado) es... ¡JSON! Y lo bueno en este caso es que podemos utilizar todo lo que hemos estado explorando anteriormente a este caso de uso sin mucho esfuerzo adicional, simplemente dándonos cuenta de que un esquema JSON no deja de ser una gramática libre de contexto. Ahora, hacer lo anterior para cada esquema que nos gustaría soportar en nuestra aplicación sería extremadamente tedioso, por lo que guidance ofrece un método de conveniencia especialmente destinado a generar JSON a partir de esquemas.

A continuación vamos a utilizar esta funcionalidad para definir generadores de personajes y de interacciones entre ellos en formatos fácilmente integrables en un entorno de juego estilo novela visual. Empecemos por la generación de personajes a partir de un esquema sencillo.

In [25]:
import json
from pydantic import BaseModel, Field, constr

from guidance import json as gen_json

# prueba a quitar esta restricción para ver cómo el modelo empieza a explayarse hasta agotar el espacio...
DescriptionStr = constr(max_length=200, pattern=r".*\.$")

# modelo de pydantic para definir un personaje
class Character(BaseModel):
    name: str = Field(max_length=20)
    description: DescriptionStr = Field(...)
    age: int = Field(gt=0, le=100)

    class Config:
        # parámetros adicionales para limpiar finales de strings y, más importantemente,
        # para evitar que el LLM añada campos no necesarios en su output.
        # Esto es relevante porque, por defecto, un esquema sólo exige la presencia de ciertos
        # campos, pero que un objeto tenga campos adicionales no invalida la especificación.
        # En generación, esto puede traducirse en un incremento significativo del coste
        # en tiempo para producir JSON, y además objetos con un exceso de datos inútiles.
        extra = 'forbid'
        str_strip_whitespace = True


lm = llama_lm

with system():
    lm += "You are a fantasy game designer focused on creating interesting yet brief character descriptions and narratives"

with user():
    n = 3
    lm += f"Create {n} characters for a disney-inspired RPG game. Each field must be one line long at most."

characters = []
with assistant():
    for i in range(n):
        lm += gen_json(name=f"character_{i}", schema=Character)
        # convertimos el string generado en un objeto de Python validado por su esquema
        result = Character.model_validate_json(lm[f"character_{i}"])
        characters.append(result)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Ahora que tenemos nuestros personajes, podemos pasar a crear una historia que haga uso de ellos.

In [26]:
from typing import Literal
from pydantic import field_validator, conlist

# tenemos la garantía de que los personajes están bien formados.
character_names = [c.name for c in characters]

class CharacterIntervention(BaseModel):
    character_name: str
    intervention_text: str = Field(max_length=200)

    class Config:
        extra = 'forbid'
        str_strip_whitespace = True

    # este es un ejemplo de cómo se pueden utilizar validadores también para guiar el output.
    # en este caso, estamos interesados en garantizar que el nombre del personaje referenciado
    # en cada una de las interacciones sea válido y esté incluído en la lista creada en el 
    # paso anterior. Esto no es necesariamente una buena práctica y de hecho el modelo no 
    # debería ser tan específico en situaciones más generales, pero aquí lo hacemos así
    # a efectos de demostrar que se pueden hacer cosas como esta.
    @field_validator('character_name')
    def name_must_be_valid(cls, v):
        if v not in character_names:
            raise ValueError(f"character_name must be one of {character_names}")
        return v
    

class Narrative(BaseModel):
    # otro ejemplo de algo que se puede hacer: establecer el rango de elementos que puede 
    # incluir una lista. De nuevo, como en el punto anterior, esto seguramente aquí no sea
    # muy adecuado si nuestra intención fuera generalizar, pero lo hacemos para demostrar
    # que se puede generar el JSON entero puramente a través del esquema y la declaración.
    interventions: conlist(CharacterIntervention, min_length=4, max_length=6)

    class Config:
        extra = 'forbid'
        str_strip_whitespace = True


# continuamos con el mismo modelo de antes (obviamos el lm = llama_lm).

with user():
    lm += f"Create a narrative dialogue between those characters."

with assistant():
    lm += gen_json(name=f"narrative", schema=Narrative)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

## Utilizando tools

El último bloque de funcionalidad de interés es la capacidad de guidance de utilizar tools externas para darle al modelo la capacidad de solicitar la realización de operaciones complejas, llamadas a APIs, o consultas a bases de datos de forma directa.

In [27]:
@guidance
def add(lm, input1, input2):
    lm += f' = {int(input1) + int(input2)}'
    return lm

@guidance
def subtract(lm, input1, input2):
    lm += f' = {int(input1) - int(input2)}'
    return lm

@guidance
def multiply(lm, input1, input2):
    lm += f' = {float(input1) * float(input2)}'
    return lm

@guidance
def divide(lm, input1, input2):
    lm += f' = {float(input1) / float(input2)}'
    return lm

lm = llama_lm

with system():
    lm += "You are a calculator bot that does not speak and only calculates."
    
with user():
    lm += "Continue the following sequence of operations.\n"
    lm += """
    1 + 1 = add(1, 1) = 2
    2 - 3 = subtract(2, 3) = -1
    """
    
with assistant():  
    lm += "5 + 6 ="
    lm += gen(max_tokens=15, stop="\n", tools=[add, subtract, multiply, divide]) + "\n"
    lm += "8 - 98 ="
    lm += gen(max_tokens=15, stop="\n", tools=[add, subtract, multiply, divide]) + "\n"
    lm += "5 * 6 ="
    lm += gen(max_tokens=15, stop="\n", tools=[add, subtract, multiply, divide]) + "\n"
    lm += "40 / 9 ="
    lm += gen(max_tokens=15, stop="\n", tools=[add, subtract, multiply, divide]) + "\n"

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

El ejemplo anterior, como ocurría con la calculadora, simplemente recurre a funciones de Python para incrementar la precisión del modelo y evitar que "prediga" algo que es fácil de resolver por código. No obstante, un caso de uso interesante es permitir que el modelo realice llamadas a APIs o bases de datos externas para acceder a información real que pueda incluir en su razonamiento, de nuevo evitando que halucine con predicciones erróneas.

En el siguiente ejemplo sencillo vamos a ver cómo utilizar tools para consultar una API muy tonta que va a permitir al modelo consultar el tiempo atmosférico para tomar decisiones informadas sobre dónde viajar.

In [28]:
import requests

city = "London"
url = f"https://wttr.in/{city}?format=3"  # formato sencillo: City: weather, temp

response = requests.get(url)
print(response.text)

London: ☀️   +25°C



In [29]:
import re

@guidance
def check_weather_in(lm, city):
    # nos limitamos a extraer el lugar y la temperatura para evitar el caracter con el estado de lluvias.
    url = f"https://wttr.in/{city}?format=3"
    response = requests.get(url)
    match = re.match(r"([^:]+):.*?([+-]?\d+°C)", response.text)
    if match:
        clean_output = f"{match.group(1)}: {match.group(2)}"
        lm += f'= {clean_output}'
    else:
        lm += "Could not parse weather data."
    return lm

lm = llama_lm

with system():
    lm += "You are a helpful reasoning agent. You solve problems by thinking step-by-step and using tools when needed."

with user():
    lm += (
        "Answer the following question by reasoning step-by-step.\n"
        "Use the format:\n"
        "Question: {question}\n"
        "Thought: {your reasoning}\n"
        "Action: {tool name}(input)\n"
        "Observation: {result from tool (check_weather_in(...))}\n"
        "... (repeat Thought/Action/Observation as needed)\n"
        "Thought: {final reasoning}\n"
        "END\n"
        "Answer: {final answer}\n\n"
        "Begin!\n\n"
        "Question: Please find a European city where temperatures are under 25 degrees now. In Madrid, it's check_weather_in(Madrid) = Madrid: +30°C. Too hot!"
    )

with assistant():
    lm += gen(
        max_tokens=10000,
        stop="END",
        tools=[check_weather_in]
    )


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

El patrón de prompting que hemos usado en la celda anterior se conoce como [ReAct Prompting](https://www.promptingguide.ai/techniques/react), un enfoque que permite a los LLMs intercalar razonamientos y acciones para planificar, adaptarse y consultar fuentes externas. Cuando esto funciona como debe, es un buen recurso para mejorar la precisión y fiabilidad de las respuestas de un chatbot. Existen muchísimas estrategias de Prompting con objetivos muy diversos. Como nos eternizaríamos hablando de todas, para un overview puedes consultar [esta referencia](https://www.promptingguide.ai/).